# 안녕하세요^^ 
# AIVLE 미니 프로젝트에 오신 여러분을 환영합니다.
* 본 과정에서는 실제 사례와 데이터를 기반으로 문제를 해결하는 전체 과정을 자기 주도형 실습으로 진행해볼 예정입니다.
* 앞선 교육과정을 정리하는 마음과 지금까지 배운 내용을 바탕으로 문제 해결을 해볼게요!
* 미니 프로젝트를 통한 문제 해결 과정 'A에서 Z까지', 지금부터 시작합니다!

---

In [1]:
# 필요 라이브러리부터 설치할께요.
!pip install pandas seaborn sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=82d8e3acb541ace7868ff5f579ba2ee43d4173209e8b535f6a9428311a5bc80d
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


---

---

# 1. 라이브러리 Import 및 데이터 불러오기

* sklearn 을 설치 하세요. 

* 관련 라이브러리들을 불러오세요

In [3]:
import pandas as pd

# 학습/평가 데이터 로딩
train_x = pd.read_csv("train_x.csv")
train_y = pd.read_csv("train_y.csv")
test_x = pd.read_csv("test_x.csv")
test_y = pd.read_csv("test_y.csv")

In [11]:
train_x.drop('time', axis = 1, inplace = True)
test_x.drop('time', axis = 1, inplace = True)

In [13]:
train_x.head()

,SO2,CO,O3,NO2,PM10,PM25,기온(°C),풍속(m/s),풍향(16방위),습도(%),...,현지기압(hPa),해면기압(hPa),전운량(10분위),중하층운량(10분위),시정(10m),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,0.003,0.6,0.002,0.039,31.0,18.0,19.2,1.3,360.0,83,...,998.8,1008.7,0.0,0,2000,17.8,22.3,22.3,22.7,22.9
1,0.003,0.6,0.002,0.035,27.0,16.0,18.7,1.0,20.0,85,...,998.8,1008.7,0.0,0,2000,17.4,22.1,22.0,22.5,22.9
2,0.003,0.6,0.002,0.033,28.0,18.0,18.3,0.3,0.0,89,...,998.9,1008.8,0.0,0,2000,17.2,21.8,21.8,22.4,22.8
3,0.003,0.6,0.002,0.030,26.0,16.0,17.7,2.0,20.0,92,...,998.1,1008.1,0.0,0,2000,17.0,21.6,21.6,22.2,22.8
4,0.003,0.5,0.003,0.026,26.0,16.0,17.4,1.0,50.0,91,...,997.9,1007.9,0.0,0,2000,16.5,21.3,21.4,22.0,22.7


#### [실습문제1] 라이브러리 불러오기



In [4]:
# 아래에 실습코드를 작성하세요.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import * 

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

---

# 2. Modeling
## 이제 모델링을 해봅시다!
+ KeyPoint : 머신러닝 라이브러리를 토대로 모델링을 할 수 있다.

### 가. LinearRegression 부터 시작해봅시다.

#### [실습문제2] 머신러닝_1
* Train과 Test로 나눈 데이터를 기준으로 LinearRegression 모델링을 진행하고 평가를 해주세요.
* 그리고 모델 파일을 pkl로 저장해주세요.
* 성능지표 : RMSE, R-squared Score(소수점 5째자리까지, 반올림)

In [26]:
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

def forward_stepwise_linear(x_train, y_train):

    # 변수목록, 선택된 변수 목록, 단계별 모델과 AIC 저장소 정의
    features = list(x_train)
    selected = []
    step_df = pd.DataFrame({ 'step':[], 'feature':[],'aic':[]})

    # 
    for s in range(0, len(features)) :
        result =  { 'step':[], 'feature':[],'aic':[]}

        # 변수 목록에서 변수 한개씩 뽑아서 모델에 추가
        for f in features :
            vars = selected + [f]
            x_tr = x_train[vars]
            model = OLS(y_train, add_constant(x_tr)).fit(disp=False)
            result['step'].append(s+1)
            result['feature'].append(vars)
            result['aic'].append(model.aic)
        
        # 모델별 aic 집계
        temp = pd.DataFrame(result).sort_values('aic').reset_index(drop = True)

        # 만약 이전 aic보다 새로운 aic 가 크다면 멈추기
        if step_df['aic'].min() < temp['aic'].min() :
            break
        step_df = pd.concat([step_df, temp], axis = 0).reset_index(drop = True)

        # 선택된 변수 제거
        v = temp.loc[0,'feature'][s]
        features.remove(v)

        selected.append(v)
    
    # 선택된 변수와 step_df 결과 반환
    return selected, step_df

In [27]:
vars, result = forward_stepwise_linear(train_x, train_y)

In [29]:
vars

['PM10', 'CO', 'PM25', 'O3', '풍향(16방위)', '습도(%)', 'NO2']

In [30]:
# 아래에 실습코드를 작성하세요.
m_lr = LinearRegression()
m_lr.fit(train_x[vars], train_y)

In [31]:
# Test 데이터로 성능을 평가해보세요.
pred_lr = m_lr.predict(test_x[vars])
print('RMSE :', mean_squared_error(test_y, pred_lr, squared=False))
print('R2_score :', r2_score(test_y, pred_lr))

RMSE :  6.041955442543057
R2_score :  0.9339541676487272


In [ ]:
# 학습한 모델을 파일로 저장해보세요.





### 나. 다음은 앙상블 기법을 토대로 랜덤포레스트와 그라디언브스팅을 활용해봅시다.

#### 1) 렌덤포레스트
##### 배깅의 일종으로 의사결정나무(Decision Tree) 여러 개를 모아서 숲을 랜덤으로 구성하고 이를 종합해서 최종 모델을 산출하는 기법이라고 할 수 있다.

#### [실습문제3] 머신러닝_2
* Train과 Test로 나눈 데이터를 기준으로 렌덤포레스트로 모델을 학습을 진행하고 평가를 해주세요.
* 그리고 모델 파일을 pkl로 저장해주세요.
* 성능지표 : RMSE, R-squared Score(소수점 5째자리까지, 반올림)

In [23]:
# 아래에 실습코드를 작성하세요.
params_rf = {'max_depth' : range(2, 8), 'n_estimators' : range(10, 100, 10)}
m_rf = RandomForestRegressor()
m_rf_gs = GridSearchCV(m_rf, params_rf, cv=5)
m_rf_gs.fit(train_x, train_y)

C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was pa

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': range(2, 8),
                         'n_estimators': range(10, 100, 10)})

In [24]:
print(m_rf_gs.best_params_, m_rf_gs.best_score_)

{'max_depth': 6, 'n_estimators': 50} 0.9079075272547584


In [25]:
# Test 데이터로 성능을 평가해보세요.
pred_rf = m_rf_gs.predict(test_x)
print('RMSE :', mean_squared_error(test_y, pred_rf, squared=False))
print('R2_score :', r2_score(test_y, pred_rf))

RMSE :  6.127207212392708
R2_score :  0.9320772099659378


In [ ]:
# 학습한 모델을 파일로 저장해보세요.





#### [실습문제4] 머신러닝_3
* 렌덤포레스트로 학습한 모델의 feature_importances 또는 Shap value를 구해보세요.
* 확인할 수 있는 내용으로 우리 모델에서의 인사이트를 정리해보세요.

In [35]:
# Feature의 중요도 확인
m_rf.fit(train_x, train_y)
m_rf.feature_importances_

<ipython-input-35-818fbe1904f6>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  m_rf.fit(train_x, train_y)


array([9.43204577e-04, 8.84739298e-04, 1.91991155e-03, 1.40823260e-03,
       9.41624970e-01, 1.34032232e-02, 9.32587159e-04, 1.48899411e-03,
       1.02057195e-03, 3.54895329e-03, 9.90642218e-04, 1.15511820e-03,
       2.82233201e-03, 2.62603895e-03, 9.52302218e-04, 2.43723272e-03,
       2.43342606e-03, 1.15246455e-03, 1.26348129e-03, 1.72047886e-03,
       5.37423713e-03, 9.89685790e-03])

In [ ]:
# 확인할 수 있는 인사이트
# 1.
# 2.
# 3.

#### 2) GradientBoosting
##### 앞선 모델의 에러를 다음 모델의 예측 값으로 활용하면서 가중치 업데이트 하는데 경사하강법(Gradient Descent)를 활용해서 최적 모델을 만드는 기법

#### [실습문제5] 머신러닝_4
* Train과 Test로 나눈 데이터를 기준으로 그라디언트부스팅으로 모델을 학습을 진행하고 평가를 해주세요.
* 그리고 모델 파일을 pkl로 저장해주세요.
* 성능지표 : RMSE, R-squared Score(소수점 5째자리까지, 반올림)

In [6]:
params_xgb = {'learning_rate' : np.linspace(0.01, 0.15, 20), 'n_estimators':range(80,150,10), 'max_depth':[3,4,5,6]}
params_xgb

{'learning_rate': array([0.01      , 0.01736842, 0.02473684, 0.03210526, 0.03947368,
        0.04684211, 0.05421053, 0.06157895, 0.06894737, 0.07631579,
        0.08368421, 0.09105263, 0.09842105, 0.10578947, 0.11315789,
        0.12052632, 0.12789474, 0.13526316, 0.14263158, 0.15      ]),
 'n_estimators': range(80, 150, 10),
 'max_depth': [3, 4, 5, 6]}

In [14]:
# 아래에 실습코드를 작성하세요.
m_xgb = XGBRegressor(objective = 'reg:squarederror')
m_xgb_gs = GridSearchCV(m_xgb, params_xgb, cv=5)
m_xgb_gs.fit(train_x, train_y)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat...
                                    predictor=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None, ...),
             param_grid={'learning_rate': array([0.01      , 0.01736842, 0.02473684, 0.03210526, 0.03947368,
       0.04684211, 0.05421053, 0.06157895, 0.06894737, 0.07631579,
       0.08368421, 0.09105263, 0.09842105, 0.10578947, 0.11315789,
       0.12052632, 0.12789474, 0.13526316, 0.14263158, 0.15      ]),
                         'max_depth': [3, 4, 5, 6],
                         'n_estimators': range(80, 150, 10)})

In [15]:
print(m_xgb_gs.best_params_, m_xgb_gs.best_score_)

{'learning_rate': 0.1057894736842105, 'max_depth': 5, 'n_estimators': 80} 0.9129538621251682


In [17]:
# Test 데이터로 성능을 평가해보세요.
pred_xgb = m_xgb_gs.predict(test_x)

In [18]:
print('RMSE :', mean_squared_error(test_y, pred_xgb, squared=False))
print('R2_score :', r2_score(test_y, pred_xgb))

RMSE :  6.210811652174674
R2_score :  0.9302109800261754


In [ ]:
# 학습한 모델을 파일로 저장해보세요.





#### [실습문제6] 머신러닝_5
* 그라디언트부스팅으로 학습한 모델의 feature_importances 또는 Shap value를 구해보세요.
* 확인할 수 있는 내용으로 우리 모델에서의 인사이트를 정리해보세요.

In [36]:
# Feature의 중요도 확인
m_xgb.fit(train_x, train_y)
m_xgb.feature_importances_

array([0.00175134, 0.0014107 , 0.00251428, 0.001899  , 0.8941148 ,
       0.0520725 , 0.00125637, 0.00196847, 0.00238518, 0.00593165,
       0.00345343, 0.00163073, 0.00277568, 0.00492355, 0.00305342,
       0.00218094, 0.00333408, 0.0017546 , 0.00270212, 0.00226639,
       0.00190619, 0.00471443], dtype=float32)

In [ ]:
# 확인할 수 있는 인사이트
# 1.
# 2.
# 3.

#### 3) Self Choice Model
##### 앞선 교육과정에서 배운 머신러닝 모델 중 어떤 것이든 좋습니다. 원하는 모델을 선택해서 학습을 시켜보세요.

#### [실습문제7] 머신러닝_6
* Train과 Test로 나눈 데이터를 기준으로 Self Choice Model로 모델을 학습을 진행하고 평가를 해주세요.
* 그리고 모델 파일을 pkl로 저장해주세요.
* 성능지표 : RMSE, R-squared Score(소수점 5째자리까지, 반올림)

In [37]:
# KNN 
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_x_s = scaler.fit_transform(train_x)
test_x_s = scaler.transform(test_x)

In [38]:
# 아래에 실습코드를 작성하세요.
params_knn = {'n_neighbors' : range(3,31,2), 'metric' : ['euclidean', 'manhattan']}
m_knn = KNeighborsRegressor()
m_knn_gs = GridSearchCV(m_knn, params_knn, cv=5)
m_knn_gs.fit(train_x_s, train_y)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': range(3, 31, 2)})

In [39]:
print(m_knn_gs.best_params_, m_knn_gs.best_score_)

{'metric': 'manhattan', 'n_neighbors': 17} 0.5279345885867046


In [40]:
# Test 데이터로 성능을 평가해보세요.
pred_knn = m_knn_gs.predict(test_x_s)
print('RMSE :', mean_squared_error(test_y, pred_knn, squared=False))
print('R2_score :', r2_score(test_y, pred_knn))

RMSE : 14.965222699869049
R2_score : 0.5948118370446835


In [ ]:
# 학습한 모델을 파일로 저장해보세요.





#### [실습문제8] 머신러닝_7
* Self Choice Model로 학습한 모델의 feature_importances 또는 Shap value를 구해보세요.
* 확인할 수 있는 내용으로 우리 모델에서의 인사이트를 정리해보세요.

In [ ]:
# 확인할 수 있는 인사이트
# 1.
# 2.
# 3.

In [42]:
# DT
params_dt = {'max_depth' : range(2,11)}
m_dt = DecisionTreeRegressor()
m_dt_gs = GridSearchCV(m_dt, params_dt, cv=5)
m_dt_gs.fit(train_x, train_y)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': range(2, 11)})

In [43]:
print(m_dt_gs.best_params_, m_dt_gs.best_score_)

{'max_depth': 6} 0.8447585688934852


In [44]:
pred_dt = m_dt_gs.predict(test_x)
print('RMSE :', mean_squared_error(test_y, pred_dt, squared=False))
print('R2_score :', r2_score(test_y, pred_dt))

RMSE : 7.053243315594216
R2_score : 0.9099946938733421


In [45]:
# SVM
params_svm = {'C' : np.linspace(50, 160, 20), 'gamma':[0.001, 0.005, 0.01, 0.05]}
m_svm = SVR()
m_svm_gs = GridSearchCV(m_svm, params_svm, cv = 5)
m_svm_gs.fit(train_x_s, train_y)

C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': array([ 50.        ,  55.78947368,  61.57894737,  67.36842105,
        73.15789474,  78.94736842,  84.73684211,  90.52631579,
        96.31578947, 102.10526316, 107.89473684, 113.68421053,
       119.47368421, 125.26315789, 131.05263158, 136.84210526,
       142.63157895, 148.42105263, 154.21052632, 160.        ]),
                         'gamma': [0.001, 0.005, 0.01, 0.05]})

In [46]:
print(m_svm_gs.best_params_, m_svm_gs.best_score_)

{'C': 160.0, 'gamma': 0.05} 0.8900631690667711


In [47]:
params_svm = {'C' : np.linspace(160, 210, 20), 'gamma':[0.05, 0.1]}
m_svm = SVR()
m_svm_gs = GridSearchCV(m_svm, params_svm, cv = 5)
m_svm_gs.fit(train_x_s, train_y)

C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': array([160.        , 162.63157895, 165.26315789, 167.89473684,
       170.52631579, 173.15789474, 175.78947368, 178.42105263,
       181.05263158, 183.68421053, 186.31578947, 188.94736842,
       191.57894737, 194.21052632, 196.84210526, 199.47368421,
       202.10526316, 204.73684211, 207.36842105, 210.        ]),
                         'gamma': [0.05, 0.1]})

In [48]:
print(m_svm_gs.best_params_, m_svm_gs.best_score_)

{'C': 210.0, 'gamma': 0.1} 0.9350103929697557


In [49]:
pred_svm = m_svm_gs.predict(test_x_s)
print('RMSE :', mean_squared_error(test_y, pred_svm, squared=False))
print('R2_score :', r2_score(test_y, pred_svm))

RMSE : 6.062872684429886
R2_score : 0.9334960748892313


In [51]:
RMSE, R2 = [],[]
model = ['LinearRegression', 'knn','dt','rf','xgb','svm']
pred = [pred_lr, pred_knn, pred_dt, pred_rf, pred_xgb, pred_svm]

for i, p in enumerate(pred) :
    RMSE.append(mean_squared_error(test_y, p, squared=False))
    R2.append(r2_score(test_y, p))
    
result = pd.DataFrame({'model':model, 'RMSE':RMSE, 'R2_score':R2})
result

,model,RMSE,R2_score
0,LinearRegression,6.041955,0.933954
1,knn,14.965223,0.594812
2,dt,7.053243,0.909995
3,rf,6.127207,0.932077
4,xgb,6.210812,0.930211
5,svm,6.062873,0.933496
